## CosmicFish v1.0

In [ ]:
#Importing main module
from cosmicfishpie.fishermatrix import cosmicfish
import os

In [ ]:
envkey = 'OMP_NUM_THREADS'
# Set this environment variable to the number of available cores in your machine, 
# to get a fast execution of the Einstein Boltzmann Solver
print("The value of {:s} is: ".format(envkey), os.environ.get(envkey))
os.environ[envkey] = str(8)
print("The value of {:s} is: ".format(envkey), os.environ.get(envkey))

## Define options and load External files. Use STEM derivatives.

In [ ]:

external = {'directory': '../../../cosmicfish_reloaded/external_input/default_camb_euclid_w0wa_HP/',  ## Files should be in the input4cast format
            'paramnames': ['Omegam', 'Omegab', 'h', 'ns', 'sigma8', 'w0','wa'],  ## Certain paramnames like Omegam and h are obligatory
            'folder_paramnames': ['Om', 'Ob', 'h', 'ns', 's8', 'w0', 'wa'],   ## Folder paramnames can have different names as paramnames
            'file_prefixes' : ['background_Hz','D_Growth-zk',  
                               'f_GrowthRate-zk', 'Plin-zk',    ## Names of cosmological quantity files can be specified here
                               'Pnonlin-zk', 'sigma8-z'],
            'k-units' : 'h/Mpc',   ## Units of the external input files
            'r-units' : 'Mpc',      
            'eps_values': [0.00625, 0.01, 0.0125, 0.01875, 0.02, 0.025, 0.03, 0.0375, 0.05, 0.10]   
            ## Epsilon parameter variations at which files were computed
            } 

fiducial = {"Omegam":0.32,
            "Omegab":0.05,
            "h":0.67,
            "ns":0.96,          ## Fiducial values of the cosmological parameters
            "sigma8":0.815584,
            "w0":-1.0,
            "wa":0.
            }
## Fiducial values of the nuisance parameters are set by default when specifying the survey below. Can be added also manually.
freepars = {"Omegam":0.01,
            "Omegab":0.01 ,
            "h":0.01,           
            "ns":0.01,        ## If derivatives are calculated with 3PT, this sets the epsilon step size, per parameter. 
            "sigma8":0.01,      ## Should match one of the epsilons available above
            "w0":0.01,
            "wa":0.01
            } 

## CosmicFish in all modes

In [ ]:
Photometric_fish_dic = dict()
Spectroscopic_fish_dic = dict()

In [ ]:
specifications = ['Euclid-ISTF-Optimistic', 'Rubin-Optimistic']

for specif in specifications:
    options = {
           'derivatives': '3PT',      ## Derivative option: 3PT or SteM
           'accuracy': 1,
           'feedback': 1,
           'code': 'external',
           'outroot': 'w0waCDM_external_camb_HP-{}'.format(specif),    #String attached to all the results files
           'results_dir' :  './results/',
           'specs_dir' : '../survey_specifications/', 
           'survey_name': specif,
           'cosmo_model' : 'w0waCDM',
           'activateMG': False
           }
    observables = ['WL','GCph']

    cosmoFM = cosmicfish.FisherMatrix(fiducialpars=fiducial,    #Pass the above dictionaries to cosmoFM, the main cosmicfish class
                              freepars=freepars,
                              options=options, 
                              observables=observables, 
                              extfiles=external, 
                              cosmoModel=options['cosmo_model'], 
                              surveyName=options['survey_name']
                              )
                            
    Photometric_fish_dic[options['outroot']] = cosmoFM.compute()    # Compute the Fisher Matrix



In [ ]:
specifications = ['Euclid-ISTF-Optimistic', 'DESI_BGS', 'DESI_ELG']
for specif in specifications:
    options = {
           'derivatives': '3PT',      ## Derivative option: 3PT or STEM of 4PT_FWD
           'accuracy': 1,
           'feedback': 1,
           'code': 'external',
           'outroot': 'w0waCDM_external_camb_HP-3PT-{}'.format(specif),    #String attached to all the results files
           'results_dir' :  './results/',
           'specs_dir' : '../survey_specifications/', 
           'survey_name': specif,
           'cosmo_model' : 'w0waCDM',
           'activateMG': False}
    observables = ['GCsp']

    cosmoFM = cosmicfish.FisherMatrix(fiducialpars=fiducial,    #Pass the above dictionaries to cosmoFM, the main cosmicfish class
                              freepars=freepars,
                              options=options, 
                              observables=observables, 
                              extfiles=external, 
                              cosmoModel=options['cosmo_model'], 
                              surveyName=options['survey_name']
                              )
                            
    Spectroscopic_fish_dic[options['outroot']] = cosmoFM.compute()    # Compute the Fisher Matrix

In [ ]:
cosmoFM.derivs_dict.keys()

# Plot the resulting Fisher matrices

In [ ]:
from cosmicfishpie.analysis import fisher_plotting as fpp
from cosmicfishpie.analysis import fisher_matrix as fm
from cosmicfishpie.analysis import fisher_operations as fo
import seaborn as sns
snscolors=sns.color_palette("colorblind")
snscolors

## Comparing the forecasts of different experiments
 - One can add Fisher matrices by just using the addition operator
 - One can marginalize over nuisance parameters
 - One can fix nuisance parameters (maximize them)

In [ ]:
Photometric_fish_dic.keys()

In [ ]:
Spectroscopic_fish_dic.keys()

### GC spectro observations: DESI ELG+BGS 

In [ ]:
Fisher_DESI_ELG = Spectroscopic_fish_dic['w0waCDM_external_camb_HP-3PT-DESI_ELG']
print(Fisher_DESI_ELG.get_param_names())
Fisher_DESI_BGS = Spectroscopic_fish_dic['w0waCDM_external_camb_HP-3PT-DESI_BGS']
print(Fisher_DESI_BGS.get_param_names())
#Fisher_DESI_EplusB = Fisher_DESI_ELG + Fisher_DESI_BGS
#Fisher_DESI_EplusB.get_param_names()

#### Nuisance parameters from each of these surveys are defined at different redshifts and values despite having same names. We need to rename them in order to be able to add them.

In [ ]:
Fisher_DESI_BGS.set_param_names?

-  We choose to reset the values for the survey with less nuisance parameters DESI_BGS

In [ ]:
Fisher_DESI_BGS.set_param_names(['Omegam', 'Omegab', 'h', 'ns', 'sigma8', 'w0', 'wa', 'lnbBgs8_1', 'lnbBgs8_2', 'lnbBgs8_3', 'lnbBgs8_4', 'lnbBgs8_5', 'PsB_1', 'PsB_2', 'PsB_3', 'PsB_4', 'PsB_5'])

- The LaTeX paramnames have been reset by this operation, so we need to set new ones

In [ ]:
print(Fisher_DESI_ELG.get_param_names_latex())

In [ ]:
Fisher_DESI_BGS.set_param_names_latex(['\\Omega_{{\\rm m}, 0}', '\\Omega_{{\\rm b}, 0}', 'h', 'n_{\\rm s}', '\\sigma_8', 'w_0', 'w_a', '\\ln(b_{B,g} \\sigma_8)_1', '\\ln(b_{B,g} \\sigma_8)_2', '\\ln(b_{B,g} \\sigma_8)_3', '\\ln(b_{B,g} \\sigma_8)_4', '\\ln(b_{B,g} \\sigma_8)_5', '\P_{B,S1}', 'P_{B,S2}', 'P_{B,S3}', 'P_{B,S4}', 'P_{B,S5}'])

#### Now we add the Fisher matrices

In [ ]:
Fisher_DESI_EB_full = Fisher_DESI_ELG + Fisher_DESI_BGS

In [ ]:
print(Fisher_DESI_EB_full.get_param_names())

#### One can also add the Fisher matrices after marginalizing or fixing the nuisance parameters, leaving only the cosmological ones

In [ ]:
fo.reshuffle?

In [ ]:
cosmoparams = ['Omegam', 'Omegab', 'h', 'ns', 'sigma8', 'w0', 'wa']

In [ ]:
Fisher_DESI_ELG_marg = fo.marginalise(Fisher_DESI_ELG, names=cosmoparams)
Fisher_DESI_ELG_fix = fo.reshuffle(Fisher_DESI_ELG, names=cosmoparams)

In [ ]:
Fisher_DESI_BGS_marg = fo.marginalise(Fisher_DESI_BGS, names=cosmoparams)
Fisher_DESI_BGS_fix = fo.reshuffle(Fisher_DESI_BGS, names=cosmoparams)

In [ ]:
Fisher_DESI_EB_marg = Fisher_DESI_ELG_marg+Fisher_DESI_BGS_marg
Fisher_DESI_EB_fix = Fisher_DESI_ELG_fix+Fisher_DESI_BGS_fix

#### Now let's compare their 1 $\sigma$ bounds

In [ ]:
plot_options = {'fishers_list': [Fisher_DESI_BGS, Fisher_DESI_ELG, Fisher_DESI_EB_marg, Fisher_DESI_EB_fix, Fisher_DESI_EB_full], 
                'colors': snscolors,
                'fish_labels': ['DESI BGS','DESI ELG', 'DESI BGS+ELG (marg. nuisance)', 'DESI BGS+ELG (fix nuisance)', 'DESI BGS+ELG (full)'],
                'filled': False,
                'plot_pars': cosmoparams,
                'axis_custom_factors': {'all':3},  ## Axis limits cover 3-sigma bounds of first Fisher matrix
                'plot_method': 'Gaussian',
                'file_format': '.pdf',   ##file format for all the plots
                'outpath' : './plots/',  ## directory where to store the files, if non-existent, it will be created
                'outroot':'DESI-GCspec_comparison_BGS_ELG_w0waCDM'  ## file name root for all the plots, extra names can be added individually
                } 

fish_plotter = fpp.fisher_plotting(**plot_options)
#fish_plotter.plot_fisher(filled=False)
fish_plotter.compare_errors(options={'yrang' : [-500, 500], 'ncol_legend': 2})

#### As expected we can see:
- BGS is a much less constraining probe than ELG
- Fixing the nuisance parameters provides the smallest errors on the full combination
- Marginalizing first over the nuisances and adding Fisher matrices is the same as adding them in full and then marginalizing over the nuisance parameters.

In [ ]:
fish_plotter.plot_fisher(filled=False)

### Combine Spectroscopic and Photometric observations

In [ ]:
Spectroscopic_fish_dic.keys()

In [ ]:
Photometric_fish_dic.keys()

In [ ]:
Fisher_Rubin_3x2photo = Photometric_fish_dic['w0waCDM_external_camb_HP-Rubin-Optimistic']
Fisher_Euclid_3x2photo = Photometric_fish_dic['w0waCDM_external_camb_HP-Euclid-ISTF-Optimistic']
Fisher_Euclid_GCspectro = Spectroscopic_fish_dic['w0waCDM_external_camb_HP-3PT-Euclid-ISTF-Optimistic']

In [ ]:
Fisher_Euclid_combined = Fisher_Euclid_GCspectro + Fisher_Euclid_3x2photo
print("Euclid Combined Fisher matrix for GC spectro + 3x2pt photometric")
for pp,ff,ss in zip(Fisher_Euclid_combined.get_param_names(), Fisher_Euclid_combined.get_param_fiducial(), Fisher_Euclid_combined.get_confidence_bounds()):
    print("Parameter name {:s}, fiducial={:.4f}, 1sigma bound: {:.2e}".format(pp,ff,ss))

In [ ]:
Fisher_RubinDESI_combined = Fisher_DESI_EB_full + Fisher_Rubin_3x2photo
print("Rubin+DESI Combined Fisher matrix for GC spectro + 3x2pt photometric")
for pp,ff,ss in zip(Fisher_RubinDESI_combined.get_param_names(), Fisher_RubinDESI_combined.get_param_fiducial(), Fisher_RubinDESI_combined.get_confidence_bounds()):
    print("Parameter name {:s}, fiducial={:.4f}, 1sigma bound: {:.2e}".format(pp,ff,ss))

In [ ]:
plot_options = {'fishers_list': [Fisher_Euclid_GCspectro, Fisher_DESI_EB_full, 
                                 Fisher_Euclid_combined, Fisher_RubinDESI_combined], 
                'colors': snscolors,
                'fish_labels': ['Euclid GCsp', 'DESI ELG+BGS GCsp', 'Euclid GCsp + 3x2photo', 'DESI ELG+BGS GCsp + Rubin 3x2photo'],
                'filled': False,
                'plot_pars': cosmoparams,
                'axis_custom_factors': {'all':3},  ## Axis limits cover 3-sigma bounds of first Fisher matrix
                'plot_method': 'Gaussian',
                'file_format': '.pdf',   ##file format for all the plots
                'outpath' : './plots/',  ## directory where to store the files, if non-existent, it will be created
                'outroot':'Euclid-Rubin-DESI-combined_w0waCDM'  ## file name root for all the plots, extra names can be added individually
                } 

fish_plotter = fpp.fisher_plotting(**plot_options)
fish_plotter.plot_fisher(filled=True)

#### Compute FoM 

In [ ]:
import numpy as np

In [ ]:
Fisher_Euclid_combined_w0wa_marg = fo.marginalise(Fisher_Euclid_combined, names=['w0', 'wa'])
print("Euclid combined total DE FoM = {:.2f}".format(np.sqrt(Fisher_Euclid_combined_w0wa_marg.determinant())))

In [ ]:
Fisher_RubinDESI_combined_w0wa_marg = fo.marginalise(Fisher_RubinDESI_combined, names=['w0', 'wa'])
print("Rubin + DESI combined total DE FoM = {:.2f}".format(np.sqrt(Fisher_RubinDESI_combined_w0wa_marg.determinant())))

In [ ]:
print(Fisher_RubinDESI_combined_w0wa_marg.fisher_matrix)
print(Fisher_Euclid_combined_w0wa_marg.fisher_matrix)

In [ ]:
plot_options = {'fishers_list': [Fisher_Euclid_combined, Fisher_RubinDESI_combined], 
                'colors': snscolors,
                'fish_labels': ['Euclid GCsp + 3x2photo', 'DESI ELG+BGS GCsp + Rubin 3x2photo'],
                'filled': False,
                'plot_pars': ['w0','wa'],
                'axis_custom_factors': {'all':3},  ## Axis limits cover 3-sigma bounds of first Fisher matrix
                'plot_method': 'Gaussian',
                'file_format': '.pdf',   ##file format for all the plots
                'outpath' : './plots/',  ## directory where to store the files, if non-existent, it will be created
                'outroot':'Euclid-Rubin-DESI-combined_w0wa-only'  ## file name root for all the plots, extra names can be added individually
                } 

fish_plotter = fpp.fisher_plotting(**plot_options)
fish_plotter.plot_fisher(filled=False)